In [36]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [37]:
import matplotlib.pyplot as plt
import os
import dateutil
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
plt.rcParams['figure.figsize'] = (10.0, 6.0)

In [38]:
def plot_memory(df):
    plt.plot(df["Time_in_s"], df["Memory"])
    plt.xlabel("Second")
    plt.xticks(rotation=30)
    plt.ylabel("Memory")
    plt.show()


In [39]:
def get_time_in_s(x):
    x = x.split("T")[-1]
    x = datetime.strptime(x, '%H:%M:%S.%f')
    a_timedelta = x - datetime(1900, 1, 1)
    seconds = a_timedelta.total_seconds()
    return seconds

def get_time_date(x):
    x = x.split("T")[-1]
    x = datetime.strptime(x, '%H:%M:%S.%f')
    return x

def locust_get_time_in_s(x):
    a_timedelta = x - datetime(1900, 1, 1)
    seconds = a_timedelta.total_seconds()
    
    return seconds 

def locust_get_time(x):
    return datetime.strptime(x, "%H:%M:%S")

def get_locust_data(data_directory):
    locust_csv = f"{data_directory}/locust_data.csv"
    df_locust = pd.read_csv(locust_csv)
    df_locust['time'] = df_locust['time'].apply(lambda x: locust_get_time(x))
    df_locust["Time_in_s"] = df_locust["time"].apply(lambda x: locust_get_time_in_s(x))
    # Start point to make everything start at zero seconds
    start_point = df_locust["Time_in_s"][0] 
    df_locust["Time_in_s"] = df_locust["Time_in_s"] -  start_point  
     
    return start_point, df_locust

def plot_pods_memory(data_directory):
    df_mem = []
    fig, ax1 = plt.subplots()
    ax1.set_title(f"Memory usage")
    ax1.set_xlabel("Second")
    ax1.set_ylabel("Memory (MiB)")
    ax1.set_xlim(0,100)
    
    for i, pod in enumerate(os.listdir(data_directory)):
        if not pod.startswith("epi-bf"):
            continue
            
        
        df_mem_pod_n = plot_mem_per_pods_and_fix_timestamps(data_directory, pod, i, ax1)
        df_mem.append(df_mem_pod_n)

    fig.tight_layout()
    plt.show()

    old_max = 0
    df_mem_total = None
        
def plot_mem_per_pods_and_fix_timestamps(data_directory, pod, i, ax):
    
    colors = ["orange", "green", "blue", "indigo", "brown"]
    
    f_name = f"{data_directory}/" + pod        
    df = pd.read_csv(f_name, delimiter=",")
    df = df.iloc[5:]
    df = df.reset_index()
    # Convert values 
    df["Time_in_s"] = df["Time"].apply(lambda x: get_time_in_s(x))
    # Locust has zero point
    df["Time_in_s"] = df["Time_in_s"] - start_point
    df["Time_datetime"] = df["Time"].apply(lambda x: get_time_date(x))
    # cpu is in nanosecond so dive by 1000 is microsecond divide by 100 is milicore
    df["Memory"] = (df["Memory"] / 1024) / 1024
    #np.gradient(df["Memory"])
    
    plot_mem_per_pod(df, pod, ax)
    
    return df

def plot_mem_per_pod(df, pod, ax):

    # Plot the cpu at a gradient of 1 precent, get microseconds of CPU per s (milicore)
    try:
        pod_name = pod.split(".")[0]
        ax.plot(df["Time_in_s"],df["Memory"], label=f"Memory {pod_name}")
        ax.legend(loc="upper left")
    except:
        print("Couldn't handle that.")

In [40]:
colors = ["r", "g", "b", "orange", "yellow", "brown"]
main_data_dir="../hpa_test_data_memory"
df_experiment = pd.read_csv("../usecases_test_vars_memory.csv")


for number_of_users in df_experiment["NO_OF_USERS"].drop_duplicates():
    df_number_of_users = df_experiment[df_experiment["NO_OF_USERS"] == number_of_users]
    fails = {}
    avg_response_50 = {}
    avg_response_95 = {}
    avg_response_50_mean = {}
    avg_response_95_mean = {}
    avg_cpu_overhead = {}
    number_of_pods = {}
    times = {}
    indexes = []
    
    
    for test_number in df_number_of_users["TEST_NO"]:
        df_number = df_number_of_users[df_number_of_users["TEST_NO"] == test_number]

        test_dir = f"TEST_NO_{test_number}"
         
#         if test_number != 1:
#             continue
        start_point, df_locust = get_locust_data(data_directory)
    
        data_directory = f"{main_data_dir}/{test_dir}"
        
      
        print(f"NUMBER_OF_USERS {number_of_users}, Test_no {test_number}")

        plot_pods_memory(data_directory)
      
    bar_width = 0.25
    start = 0.5
    offset = 0.25

   TEST_NO  NO_OF_USERS  SPAWN_RATE  RUN_TIME  HPA_MAX_REPLICAS  \
0        1           10           1       100                 5   
1        2          100          10       100                 5   
2        3         1000         100       100                 5   
3        4        10000        1000       100                 5   

   HPA_UTILIZATION CPU_LIMIT Memory_LIMIT  
0               30     1000m        500Mi  
1               30     1000m        500Mi  
2               30     1000m        500Mi  
3               30     1000m        500Mi  


FileNotFoundError: [Errno 2] No such file or directory: '../hpa_test_data_memory/TEST_NO_5/locust_data.csv'